In [ ]:
!pip install duckdb pandas

In [1]:
# Duckdb is a SQL engine that allows us to execute powerful, analytics-friendly
# queries against local or remote databases and flat files.
import duckdb
import pandas as pd

# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
# Create a database file on disk
conn = duckdb.connect('example.db')
# Enable remote access
conn.sql("INSTALL httpfs")
conn.sql("LOAD httpfs")
# This database file points to files totaling multiple GBs,
# but it's only about 300KB itself. The `ATTACH` command
# gives us access to views that sit on top of remote Parquet files.
try:
  conn.sql("ATTACH 'https://data.baseball.computer/dbt/bc_remote.db' (READ_ONLY)")
except duckdb.BinderException:
  # This command will fail if you run it more than once because it already exists,
  # in which case we don't need to do anything
  pass

conn.sql("USE bc_remote")
conn.sql("USE main_models")

Potential A/B Testing ideas: Offensive and pitching stats several years prior and post expansion
    Offensive and Defensive stats post DH expansion
    Offensive stats post pitch clock
    
Explore active player's career homerun totals and what percentage of them were on their current teams. 

In [7]:
df: pd.DataFrame = conn.sql("""WITH players AS 
                            (SELECT DISTINCT
                                r.player_id, 
                                r.last_name, 
                                r.first_name 
                            FROM misc.roster r  
                            WHERE r.year = 2023 
                            ), 
                            offStats AS ( 
                            SELECT DISTINCT
                                p.player_id, 
                                p.last_name, 
                                p.first_name, 
                                --pgo.game_id,
                                sum(ebs.hits) AS hits,
                                sum(ebs.home_runs) AS HRs
                            FROM event_batting_stats ebs 
                            left join players p on ebs.batter_id = p.player_id 
                            left join stg_gamelog gl on ebs.game_id = gl.game_id
                            where gl.season = 2023
                            GROUP BY p.player_id, p.last_name, p.first_name
                            )
                            SELECT * FROM offStats;""").df()
    
df.sort_values('hits', ascending = False)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,player_id,last_name,first_name,hits,HRs
144,acunr001,Acuna,Ronald,219.0,41.0
514,freef001,Freeman,Freddie,212.0,29.0
506,arral001,Arraez,Luis,206.0,10.0
268,semim001,Semien,Marcus,202.0,31.0
507,turnt001,Turner,Trea,187.0,29.0
...,...,...,...,...,...
303,shewb001,Shewmake,Braden,0.0,0.0
467,sborj002,Sborz,Josh,0.0,0.0
629,bannr001,Bannon,Rylan,0.0,0.0
579,mitcc002,Mitchell,Cal,0.0,0.0


In [7]:
df.groupby('player_id').sum('home_runs')

,hits,HRs
player_id,,
abrac001,138.0,18.0
abrej003,141.0,22.0
abrew002,24.0,2.0
acunr001,219.0,41.0
adamj003,5.0,0.0
...,...,...
yoshm002,155.0,15.0
younj003,8.0,2.0
younj004,27.0,0.0


In [51]:
df: pd.DataFrame = conn.sql("""SELECT 
                                pgo.*
                                --(pgo.hits) AS totalHits, 
                                --sum(pgo.home_runs) AS totalHRs 
                            FROM event_batting_stats pgo 
                            left join misc.roster r on pgo.batter_id = r.player_id 
                            left join stg_gamelog gl on pgo.game_id = gl.game_id
                            where pgo.batter_id = 'sancg002' AND gl.season = 2023 AND pgo.game_id = 'SDN202308180';""").df()

df

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,game_id,event_key,batter_id,pitcher_id,batting_team_id,fielding_team_id,batter_lineup_position,plate_appearances,at_bats,hits,...,on_base_opportunities,on_base_successes,runs_batted_in,grounded_into_double_plays,double_plays,triple_plays,batting_outs,outs_on_play,left_on_base,left_on_base_with_two_outs
0,SDN202308180,654600828,sancg002,pfaab001,SDN,ARI,7,1,1,0,...,1,0,0,0,0,0,1,1,0,0
1,SDN202308180,654600849,sancg002,pfaab001,SDN,ARI,7,1,1,0,...,1,0,0,0,0,0,1,1,1,1
2,SDN202308180,654600871,sancg002,castm002,SDN,ARI,7,1,0,0,...,1,1,0,0,0,0,0,0,0,0
3,SDN202308180,654600828,sancg002,pfaab001,SDN,ARI,7,1,1,0,...,1,0,0,0,0,0,1,1,0,0
4,SDN202308180,654600849,sancg002,pfaab001,SDN,ARI,7,1,1,0,...,1,0,0,0,0,0,1,1,1,1
5,SDN202308180,654600871,sancg002,castm002,SDN,ARI,7,1,0,0,...,1,1,0,0,0,0,0,0,0,0
6,SDN202308180,654600828,sancg002,pfaab001,SDN,ARI,7,1,1,0,...,1,0,0,0,0,0,1,1,0,0
7,SDN202308180,654600849,sancg002,pfaab001,SDN,ARI,7,1,1,0,...,1,0,0,0,0,0,1,1,1,1
8,SDN202308180,654600871,sancg002,castm002,SDN,ARI,7,1,0,0,...,1,1,0,0,0,0,0,0,0,0
9,SDN202308180,654600828,sancg002,pfaab001,SDN,ARI,7,1,1,0,...,1,0,0,0,0,0,1,1,0,0


In [55]:
df: pd.DataFrame = conn.sql("""SELECT DISTINCT player_id, game_id, COUNT(*)
                                FROM player_game_offense_stats
                                GROUP BY player_id, game_id
                                HAVING COUNT(*) > 1""").df()

df

,player_id,game_id,count_star()
0,radca101,CAG194209072,2


In [27]:
df: pd.DataFrame = conn.sql("""SELECT DISTINCT player_id, first_name, last_name, COUNT(*)
                                FROM misc.roster
                                where year = 2023 
                                GROUP BY player_id, first_name, last_name
                                HAVING COUNT(*) > 1
                                """).df()
df

,player_id,first_name,last_name,count_star()
0,tapir001,Raimel,Tapia,3
1,sweed002,Devin,Sweet,2
2,badeh001,Harrison,Bader,2
3,hatct001,Thomas,Hatch,2
4,heart001,Taylor,Hearn,3
...,...,...,...,...
146,rookb001,Brent,Rooker,2
147,gioll001,Lucas,Giolito,3
148,weisz001,Zack,Weiss,2
149,rojaj001,Josh,Rojas,2


In [31]:
df: pd.DataFrame = conn.sql("""SELECT *
                                FROM misc.roster
                                where year = 2023 and last_name = 'Alonso'
                                """).df()
df

,year,player_id,last_name,first_name,bats,throws,team_id,position
0,2023,alonp001,Alonso,Pete,R,R,NYN,PH
1,2023,alonp001,Alonso,Pete,R,R,NYN,1B


For code below
1. SUM(CASE WHEN sequence_item = 'Ball' THEN 1 ELSE 0 END) OVER (...):

    This is the window function that calculates the cumulative sum of occurrences of 'Ball' for each sequence_id.
    It does this within the partition of game_id and event_key and orders the data by sequence_id.
    The result is an incremental count of 'Ball' values as it processes each row, creating the behavior you described (e.g., 1, 2, 3, 3, 4).

2. PARTITION BY game_id, event_key: Ensures that the counting restarts for each unique combination of game_id and event_key.

3. ORDER BY sequence_id: Ensures that the Ball_count increments in the correct order based on sequence_id.

4. GROUP BY: We use GROUP BY to include other counts (for SwingStrike and CalledStrike) while still showing the incremental Ball_count for each sequence_id.

In [23]:
df: pd.DataFrame = conn.sql("""WITH PitchCount AS (select
                                                        game_id,
                                                        event_key,
                                                        sequence_id,
                                                        sequence_item,
                                                        SUM(CASE WHEN sequence_item IN ('AutomaticBall',
                                                                                        'Ball',
                                                                                        'Pitchout',
                                                                                        'IntentionalBall') THEN 1 ELSE 0 END)
                                                            OVER(PARTITION BY game_id, event_key order by sequence_id) AS BallCount,
                                                        SUM(CASE WHEN sequence_item IN ('CalledStrike', 
                                                                                        'MissedBunt', 
                                                                                        'StrikeUnknownType', 
                                                                                        'SwingingOnPitchout', 
                                                                                        'SwingingStrike') THEN 1 
                                                            ELSE 0 
                                                        END)
                                                            OVER(PARTITION BY game_id, event_key order by sequence_id) AS StrikeCount
                                                    FROM event.event_pitch_sequences
                                                    where 
                                                        game_id = 'SDN202308180'
                                                        --AND event_key = '654600812'
                                                        
                                    )
                                    select game_id, 
                                        event_key, 
                                        sequence_id, 
                                        BallCount,
                                        SUM(CASE
                                                WHEN sequence_item IN ('CalledStrike', 
                                                                        'MissedBunt', 
                                                                        'StrikeUnknownType', 
                                                                        'SwingingOnPitchout', 
                                                                        'SwingingStrike') THEN 1 
                                                WHEN sequence_item IN ('Foul',
                                                                        'FoulBunt', 
                                                                        'FoulOnPitchout', 
                                                                        'FoulTip', 
                                                                        'FoulTipBunt') 
                                                                        AND StrikeCount < 2 THEN 1 
                                                ELSE 0 
                                            END)
                                        OVER(PARTITION BY game_id, event_key order by sequence_id) AS StrikeCount,
                                        COUNT(CASE WHEN sequence_item IN ('AutomaticBall',
                                                                            'Ball',
                                                                            'Pitchout',
                                                                            'IntentionalBall') THEN 1 END) AS Ball, 
                                        count(CASE WHEN sequence_item = 'SwingStrike' THEN 1 END) AS swingStrike, 
                                        count(CASE WHEN sequence_item = 'CalledStrike' THEN 1 END) AS CalledStrike,
                                        count(CASE WHEN sequence_item IN ('CalledStrike', 
                                                                            'MissedBunt', 
                                                                            'StrikeUnknownType', 
                                                                            'SwingingOnPitchout', 
                                                                            'SwingingStrike',
                                                                            'Foul',
                                                                            'FoulBunt', 
                                                                            'FoulOnPitchout', 
                                                                            'FoulTip', 
                                                                            'FoulTipBunt') 
                                                          THEN 1 END) AS Strike 
                    from PitchCount 
                    where game_id = 'SDN202308180' AND StrikeCount >= 2
                    GROUP BY 
                        game_id, event_key, sequence_id, BallCount, StrikeCount
                    order by event_key
                            """).df()
df

#'HitBatter', 
#'InPlay', 
#'InPlayOnPitchout', 


#'NoPitch', 
#'PickoffAttemptFirst', 
#'PickoffAttemptSecond', 
#'PickoffAttemptThird', 
###



BinderException: Binder Error: column "sequence_item" must appear in the GROUP BY clause or must be part of an aggregate function.
Either add it to the GROUP BY list, or use "ANY_VALUE(sequence_item)" if the exact value of "sequence_item" is not important.

In [39]:
df: pd.DataFrame = conn.sql("""SELECT * FROM event_offense_stats where event_key = '654600812' """).df()

df

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,game_id,event_key,team_id,player_id,baserunner,plate_appearances,at_bats,hits,singles,doubles,triples,home_runs,total_bases,strikeouts,walks,intentional_walks,hit_by_pitches,sacrifice_hits,sacrifice_flies,reached_on_errors,reached_on_interferences,inside_the_park_home_runs,ground_rule_doubles,infield_hits,on_base_opportunities,on_base_successes,runs_batted_in,grounded_into_double_plays,double_plays,triple_plays,batting_outs,balls_in_play,balls_batted,trajectory_fly_ball,trajectory_ground_ball,trajectory_line_drive,trajectory_pop_up,trajectory_unknown,trajectory_known,trajectory_broad_air_ball,trajectory_broad_ground_ball,trajectory_broad_unknown,trajectory_broad_known,bunts,batted_distance_plate,batted_distance_infield,batted_distance_outfield,batted_distance_unknown,batted_distance_known,fielded_by_battery,fielded_by_infielder,fielded_by_outfielder,fielded_by_known,fielded_by_unknown,batted_angle_left,batted_angle_right,batted_angle_middle,batted_angle_unknown,batted_angle_known,batted_location_plate,batted_location_right_infield,batted_location_middle_infield,batted_location_left_infield,batted_location_left_field,batted_location_center_field,batted_location_right_field,batted_location_unknown,batted_location_known,batted_balls_pulled,batted_balls_opposite_field,runs,times_reached_base,stolen_bases,caught_stealing,picked_off,picked_off_caught_stealing,outs_on_basepaths,unforced_outs_on_basepaths,outs_avoided_on_errors,advances_on_wild_pitches,advances_on_passed_balls,advances_on_balks,advances_on_unspecified_plays,advances_on_defensive_indifference,advances_on_errors,plate_appearances_while_on_base,balls_in_play_while_running,balls_in_play_while_on_base,batter_total_bases_while_running,batter_total_bases_while_on_base,extra_base_advance_attempts,bases_advanced,bases_advanced_on_balls_in_play,surplus_bases_advanced_on_balls_in_play,outs_on_extra_base_advance_attempts,pitches,swings,swings_with_contact,strikes,strikes_called,strikes_swinging,strikes_foul,strikes_foul_tip,strikes_in_play,strikes_unknown,balls,balls_called,balls_intentional,balls_automatic,unknown_pitches,pitchouts,pitcher_pickoff_attempts,catcher_pickoff_attempts,pitches_blocked_by_catcher,pitches_with_runners_going,passed_balls,wild_pitches,balks,left_on_base,left_on_base_with_two_outs,stolen_bases_second,stolen_bases_third,stolen_bases_home,caught_stealing_second,caught_stealing_third,caught_stealing_home,stolen_base_opportunities,stolen_base_opportunities_second,stolen_base_opportunities_third,stolen_base_opportunities_home,picked_off_first,picked_off_second,picked_off_third,times_force_on_runner,times_lead_runner,times_next_base_empty,extra_base_chances,extra_bases_taken
0,SDN202308180,654600812,ARI,martk001,Batter,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,5,0,0,1,1,0,0,0,0,0,4,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0


In [59]:
conn.close()